In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.signal import resample
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [2]:

train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
sub = pd.read_csv('../input/submit.csv')
y = train.groupby('fragment_id')['behavior_id'].min()

In [3]:
train['mod'] = (train.acc_x ** 2 + train.acc_y ** 2 + train.acc_z ** 2) ** .5
train['modg'] = (train.acc_xg ** 2 + train.acc_yg ** 2 + train.acc_zg ** 2) ** .5
test['mod'] = (test.acc_x ** 2 + test.acc_y ** 2 + test.acc_z ** 2) ** .5
test['modg'] = (test.acc_xg ** 2 + test.acc_yg ** 2 + test.acc_zg ** 2) ** .5

In [4]:
x = np.zeros((7292, 60, 8, 1))
t = np.zeros((7500, 60, 8, 1))
for i in tqdm(range(7292)):
    tmp = train[train.fragment_id == i][:60]
    x[i,:,:, 0] = resample(tmp.drop(['fragment_id', 'time_point', 'behavior_id'],
                                    axis=1), 60, np.array(tmp.time_point))[0]
for i in tqdm(range(7500)):
    tmp = test[test.fragment_id == i][:60]
    t[i,:,:, 0] = resample(tmp.drop(['fragment_id', 'time_point'],
                                    axis=1), 60, np.array(tmp.time_point))[0]

100%|█████████████████████████████████████████████████████████████████████████████| 7500/7500 [00:11<00:00, 632.55it/s]


In [5]:
def Net():
    input = Input(shape=(60, 8, 1))
    X = Conv2D(filters=64,
               kernel_size=(3, 3),
               activation='relu',
               padding='same')(input)
    X = Conv2D(filters=128,
               kernel_size=(3, 3),
               activation='relu',
               padding='same')(X)
    X = MaxPooling2D()(X)
    X = Conv2D(filters=256,
               kernel_size=(3, 3),
               activation='relu',
               padding='same')(X)
    X = Conv2D(filters=512,
               kernel_size=(3, 3),
               activation='relu',
               padding='same')(X)
    X = GlobalMaxPooling2D()(X)
    X = Dropout(0.3)(X)
    X = Dense(19, activation='softmax')(X)
    return Model([input], X)

In [6]:
kfold = StratifiedKFold(5, shuffle=True)

In [7]:
proba_t = np.zeros((7500, 19))
for fold, (xx, yy) in enumerate(kfold.split(x, y)):
    y_ = to_categorical(y, num_classes=19)
    model = Net()
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(),
                  metrics=['acc'])
    plateau = ReduceLROnPlateau(monitor="val_acc",
                                verbose=0,
                                mode='max',
                                factor=0.1,
                                patience=6)
    early_stopping = EarlyStopping(monitor='val_acc',
                                   verbose=0,
                                   mode='max',
                                   patience=10)
    checkpoint = ModelCheckpoint(f'fold{fold}.h5',
                                 monitor='val_acc',
                                 verbose=0,
                                 mode='max',
                                 save_best_only=True)
    model.fit(x[xx], y_[xx],
              epochs=100,
              batch_size=512,
              verbose=1,
              shuffle=True,
              validation_data=(x[yy], y_[yy]),
              callbacks=[plateau, early_stopping, checkpoint])
    model.load_weights(f'fold{fold}.h5')
    proba_t += model.predict(t, verbose=0, batch_size=1024) / 5.

Train on 5833 samples, validate on 1459 samples
Epoch 1/100
5833/5833 [==============================] - 19s 3ms/sample - loss: 2.6260 - acc: 0.2050 - val_loss: 2.2182 - val_acc: 0.3379
Epoch 2/100
5833/5833 [==============================] - 6s 1ms/sample - loss: 2.0424 - acc: 0.3305 - val_loss: 1.9371 - val_acc: 0.3475
Epoch 3/100
5833/5833 [==============================] - 6s 1ms/sample - loss: 1.8397 - acc: 0.3832 - val_loss: 1.7376 - val_acc: 0.4106
Epoch 4/100
5833/5833 [==============================] - 6s 1ms/sample - loss: 1.6834 - acc: 0.4187 - val_loss: 1.6349 - val_acc: 0.4524
Epoch 5/100
5833/5833 [==============================] - 6s 1ms/sample - loss: 1.5761 - acc: 0.4519 - val_loss: 1.5666 - val_acc: 0.4798
Epoch 6/100
5833/5833 [==============================] - 6s 1ms/sample - loss: 1.5224 - acc: 0.4605 - val_loss: 1.5328 - val_acc: 0.4654
Epoch 7/100
5833/5833 [==============================] - 6s 1ms/sample - loss: 1.4444 - acc: 0.4857 - val_loss: 1.4634 - val_acc:

5833/5833 [==============================] - 6s 1ms/sample - loss: 0.3149 - acc: 0.8908 - val_loss: 1.1346 - val_acc: 0.6854
Epoch 61/100
5833/5833 [==============================] - 6s 1ms/sample - loss: 0.3016 - acc: 0.9016 - val_loss: 1.1255 - val_acc: 0.6875
Epoch 62/100
5833/5833 [==============================] - 6s 1ms/sample - loss: 0.3020 - acc: 0.9013 - val_loss: 1.1327 - val_acc: 0.6875
Epoch 63/100
5833/5833 [==============================] - 6s 1ms/sample - loss: 0.2957 - acc: 0.9021 - val_loss: 1.1304 - val_acc: 0.6868
Epoch 64/100
5833/5833 [==============================] - 6s 1ms/sample - loss: 0.2973 - acc: 0.9047 - val_loss: 1.1302 - val_acc: 0.6888
Epoch 65/100
5833/5833 [==============================] - 6s 1ms/sample - loss: 0.2979 - acc: 0.9004 - val_loss: 1.1305 - val_acc: 0.6895
Epoch 66/100
5833/5833 [==============================] - 6s 1ms/sample - loss: 0.2948 - acc: 0.9013 - val_loss: 1.1303 - val_acc: 0.6888
Train on 5833 samples, validate on 1459 samples

5833/5833 [==============================] - 6s 1ms/sample - loss: 0.3477 - acc: 0.8776 - val_loss: 1.1341 - val_acc: 0.6779
Epoch 54/100
5833/5833 [==============================] - 6s 1ms/sample - loss: 0.3464 - acc: 0.8814 - val_loss: 1.1386 - val_acc: 0.6840
Epoch 55/100
5833/5833 [==============================] - 6s 1ms/sample - loss: 0.3371 - acc: 0.8824 - val_loss: 1.1412 - val_acc: 0.6854
Epoch 56/100
5833/5833 [==============================] - 6s 1ms/sample - loss: 0.3303 - acc: 0.8911 - val_loss: 1.1376 - val_acc: 0.6902
Epoch 57/100
5833/5833 [==============================] - 6s 1ms/sample - loss: 0.3317 - acc: 0.8863 - val_loss: 1.1458 - val_acc: 0.6868
Epoch 58/100
5833/5833 [==============================] - 6s 1ms/sample - loss: 0.3242 - acc: 0.8918 - val_loss: 1.1517 - val_acc: 0.6833
Epoch 59/100
5833/5833 [==============================] - 6s 1ms/sample - loss: 0.3211 - acc: 0.8913 - val_loss: 1.1521 - val_acc: 0.6881
Epoch 60/100
5833/5833 [=======================

5834/5834 [==============================] - 6s 1ms/sample - loss: 0.5066 - acc: 0.8183 - val_loss: 1.1047 - val_acc: 0.6578
Epoch 40/100
5834/5834 [==============================] - 6s 1ms/sample - loss: 0.5011 - acc: 0.8241 - val_loss: 1.1185 - val_acc: 0.6536
Epoch 41/100
5834/5834 [==============================] - 6s 1ms/sample - loss: 0.4905 - acc: 0.8250 - val_loss: 1.0774 - val_acc: 0.6687
Epoch 42/100
5834/5834 [==============================] - 6s 1ms/sample - loss: 0.4729 - acc: 0.8286 - val_loss: 1.1498 - val_acc: 0.6598
Epoch 43/100
5834/5834 [==============================] - 6s 1ms/sample - loss: 0.4625 - acc: 0.8334 - val_loss: 1.1111 - val_acc: 0.6708
Epoch 44/100
5834/5834 [==============================] - 6s 1ms/sample - loss: 0.4521 - acc: 0.8358 - val_loss: 1.2024 - val_acc: 0.6783
Epoch 45/100
5834/5834 [==============================] - 6s 1ms/sample - loss: 0.4108 - acc: 0.8533 - val_loss: 1.1085 - val_acc: 0.6763
Epoch 46/100
5834/5834 [=======================

5834/5834 [==============================] - 6s 1ms/sample - loss: 0.7604 - acc: 0.7249 - val_loss: 1.0803 - val_acc: 0.6351
Epoch 30/100
5834/5834 [==============================] - 6s 1ms/sample - loss: 0.7334 - acc: 0.7329 - val_loss: 1.0794 - val_acc: 0.6351
Epoch 31/100
5834/5834 [==============================] - 6s 1ms/sample - loss: 0.7214 - acc: 0.7359 - val_loss: 1.0589 - val_acc: 0.6406
Epoch 32/100
5834/5834 [==============================] - 6s 1ms/sample - loss: 0.6983 - acc: 0.7455 - val_loss: 1.0433 - val_acc: 0.6475
Epoch 33/100
5834/5834 [==============================] - 6s 1ms/sample - loss: 0.6754 - acc: 0.7554 - val_loss: 1.0817 - val_acc: 0.6379
Epoch 34/100
5834/5834 [==============================] - 6s 1ms/sample - loss: 0.6679 - acc: 0.7518 - val_loss: 1.0556 - val_acc: 0.6488
Epoch 35/100
5834/5834 [==============================] - 6s 1ms/sample - loss: 0.6611 - acc: 0.7660 - val_loss: 1.0530 - val_acc: 0.6578
Epoch 36/100
5834/5834 [=======================

5834/5834 [==============================] - 6s 1ms/sample - loss: 1.5718 - acc: 0.4498 - val_loss: 1.4274 - val_acc: 0.4822
Epoch 7/100
5834/5834 [==============================] - 6s 1ms/sample - loss: 1.4988 - acc: 0.4700 - val_loss: 1.3530 - val_acc: 0.4931
Epoch 8/100
5834/5834 [==============================] - 6s 1ms/sample - loss: 1.3975 - acc: 0.4991 - val_loss: 1.3064 - val_acc: 0.5254
Epoch 9/100
5834/5834 [==============================] - 6s 1ms/sample - loss: 1.3659 - acc: 0.5072 - val_loss: 1.3035 - val_acc: 0.5343
Epoch 10/100
5834/5834 [==============================] - 6s 1ms/sample - loss: 1.3233 - acc: 0.5305 - val_loss: 1.2248 - val_acc: 0.5466
Epoch 11/100
5834/5834 [==============================] - 6s 1ms/sample - loss: 1.2846 - acc: 0.5418 - val_loss: 1.2071 - val_acc: 0.5583
Epoch 12/100
5834/5834 [==============================] - 6s 1ms/sample - loss: 1.2511 - acc: 0.5514 - val_loss: 1.1666 - val_acc: 0.5775
Epoch 13/100
5834/5834 [==========================

In [8]:
sub.behavior_id = np.argmax(proba_t, axis=1)
sub.to_csv('../sub/submit_baseline_nn.csv', index=False)